<a target="_blank" href="https://colab.research.google.com/github/trainocate-japan/Machine-Learning-and-Deep-Learning-Hands-on/blob/main/exercise/6_ディープラーニング/6-3_（演習）TensorFlow_Kerasによるワインの品種分類.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 6-3_TensorFlow/Kerasによるワインの品種分類
このノートブックでは、TensorFlow / Kerasで分類の予測モデルを作成します。<br>
予測を行うテーマはwineの成分からブドウの品種を予測することです。

## ライブラリのインポート

In [ ]:
# データを処理するための基本的なライブラリ
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

# scikit-learnから必要なライブラリをインポート
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# TensorFlow/Kerasで使用
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping # 早期終了

Google Colaboratory上での出力のデフォルト設定

In [ ]:
# pandasのDataframeの出力
pd.set_option('display.max_columns', 500) # 表示列の最大
pd.set_option('display.max_rows', 500) # 表示行の最大
pd.set_option('display.unicode.east_asian_width', True) # 日本語出力時にヘッダのずれを解消
pd.options.display.float_format = '{:,.5f}'.format # 表示桁数の設定

# ノートブックの表示桁数設定。この設定はprint文には作用せず、セルの最後に書いたものを出力する際に適用されます。
%precision 3
# numpy配列の指数表示禁止設定
np.set_printoptions(suppress=True)
# numpy配列の表示桁数設定
np.set_printoptions(precision=3)

## データの準備
今回使用するデータはscikit-learnからもデータセットとして利用することができる、UCI ML Wine Data Setのコピーです。<br>
UCI Machine Learning Repositoryから公開されています。<br>
downloaded from : https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data

データセットについての説明はこちらに記載されています。<br>
https://scikit-learn.org/stable/datasets/toy_dataset.html#wine-dataset

#### データを取り込む
- pandasのread_csvメソッドを使用して、mlho/data/wine.csvファイルを読み込みます
- 読み込んだものは変数df_wineに代入します

In [ ]:
# ★★wine.csv を変数df_wineに読み込みます


#### データを確認する
*   Class以外：ワインの成分を表す値
*   Class：ブドウの品種を0,1,2であらわしている



In [ ]:
# ★★読み込んだデータをheadメソッドで確認します


In [ ]:
# ★★df_wineのデータ要約を確認


In [ ]:
# ★★df_wineの統計情報を確認


#### 説明変数と目的変数を切り出す

In [ ]:
# dropメソッドを使って、目的変数にするClass以外をすべて説明変数xに代入する
x = df_wine.drop(columns='Class')
x.tail(4)

In [ ]:
# ★★tailメソッドでxの中身を確認する


In [ ]:
# 目的変数はClass
# copy()を使用して、df_wineからClassの列のコピーを取得
# (後でDataFrameの列を増やすため、参照渡しを防ぐ目的）
#★y = df_wine[['Class']].copy()
#★y.tail(4)

#### 目的変数をone-hot表現に変換する

In [ ]:
# Kerasの多値分類では目的変数をone-hot表現に直します
# pop('列名')でその列の値だけを取り出します(もとのDataFrameからは列が消去されます)
Class = y.pop('Class')

In [ ]:
# 0,1,2それぞれをclass_0, class_1, class_2のone-hot表現に変換します
# booleanで値が帰ってきているため1を掛けてintにしています
y['class_0'] = (Class == 0)*1
y['class_1'] = (Class == 1)*1
y['class_2'] = (Class == 2)*1
y.tail(4)

#### データを訓練データと検証データに分割する

In [ ]:
# ★★訓練データと検証データに分割（80%を訓練用に使用、random_state=0を指定）


### データのスケールを揃える

説明変数を標準化

In [ ]:
# 訓練データ説明変数の各列の平均を計算する
#train_x_mean = train_x.mean()
train_x_mean.head()

In [ ]:
# 訓練データ説明変数の各列の標準偏差を計算する
train_x_std = train_x.std()
train_x_std.head()

In [ ]:
# 訓練データ説明変数の標準化を行う
train_x_scaled = (train_x - train_x_mean) / train_x_std
train_x_scaled.head()

In [ ]:
# 検証データ説明変数の標準化を行う
val_x_scaled = (val_x - train_x_mean) / train_x_std
val_x_scaled.head()

## モデルの定義

In [ ]:
tf.random.set_seed(0)

# モデルオブジェクトを用意し必要な層を追加していく
model = Sequential()

# ★★中間層1層目（64ノード、活性化関数はrelu関数）


# ★★中間層2層目（64ノード、活性化関数はrelu関数）


# ★★出力層（多値分類なので、3ノード、活性化関数はsoftmax関数）


# 最適化手法としてAdam、誤差関数として交差エントロピー誤差を設定
# 分類の場合、metricsに正解率を指定することで、エポックごとに正解率も計算させることができる
#★optimizer = optimizers.Adam(learning_rate=0.001)
#★model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])

model.summary()

## モデルの学習

In [ ]:
# ★★定義したモデルで学習を行う（エポック数700、バッチサイズ32、validation_data=(val_x_scaled, val_y)戻り値は変数historyに格納）


## 評価

In [ ]:
# ★★学習状況を可視化する
# 今回は学習過程で損失、正解率ともに確認しているので、可視化のみ行う


## ニューラルネットワークモデルを改良する

In [ ]:
tf.random.set_seed(0)

model = Sequential()
# 中間層1
model.add(Dense(64, input_shape=(train_x.shape[1],)))
model.add(Activation('relu')) 
                         # ★★ここにドロップアウトを追加
# 中間層2
model.add(Dense(64))
model.add(Activation('relu')) 
                         # ★★ここにドロップアウトを追加
# 出力層
model.add(Dense(3))
model.add(Activation('softmax'))
# 最適化手法としてAdam、誤差関数として交差エントロピー誤差を設定
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss',
                       patience=10,
                       verbose=1)

In [ ]:
history = model.fit(train_x_scaled, train_y, epochs=700, validation_data = (val_x_scaled, val_y), verbose=1) # ★★ここを変更してEarlyStoppingを設定

In [ ]:
# ★★学習状況を可視化する


※ ニューラルネットワークの構成によっては、結果が97.2%もしくは100%になるかもしれません。これはテストデータを切り出す際に全体の20%で指定をすると36件のデータがテストデータとなり、そのうちの35件を正しく予測できると97.2%になるため、35件あてられたか、36件あてられたかの違いです。

このノートブックは以上です。